## Step-by-Step Guide



#### Import Libraries


In [2]:
# from PIL import Image
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence


#### Set Up Data Preprocessing


In [3]:
class DataGenerator(Sequence):
    def __init__(self, images_dir, labels_csv, batch_size=32, image_size=(224, 224), shuffle=True):
        self.images_dir = images_dir
        self.labels_df = pd.read_csv(labels_csv)
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.image_ids = self.labels_df['image'].values
        self.labels = self.labels_df['count'].values
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        image_ids_batch = [self.image_ids[k] for k in indexes]
        labels_batch = [self.labels[k] for k in indexes]
        
        images = np.array([self.load_image(img_id) for img_id in image_ids_batch])
        labels = np.array(labels_batch).astype('float32')
        
        return images, labels

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def load_image(self, img_id):
        img_path = os.path.join(self.images_dir, img_id)
        img = Image.open(img_path).resize(self.image_size)
        img = np.array(img) / 255.0
        return img


In [4]:
train_generator = DataGenerator('model_data/train/images', 'model_data/train/output_label.csv')
val_generator = DataGenerator('model_data/val/images', 'model_data/val/output_label.csv')
test_generator = DataGenerator('model_data/test/images', 'model_data/test/output_label.csv')


#### Define Model Architecture


In [5]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='linear')  # Single output for regression
])


C:\Users\karma\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Compile the Model


In [6]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',  # MSE for regression
    metrics=['mae']  # MAE is helpful for interpretability in regression
)



#### Train the Model




In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping]
)


Epoch 1/10


C:\Users\karma\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/50 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 536.6505 - mae: 19.3038

#### Evaluate the Model


In [ ]:
test_loss, test_mae = model.evaluate(test_generator)
print(f"Test MAE: {test_mae}")


#### Save the Trained Model

In [ ]:
pip uninstall h5py


In [ ]:
# Save only the weights
model.save_weights('model_weights')

# Later, recreate the model and load weights
model = ...  # Define your model architecture here
model.load_weights('model_weights')
